In [1]:
%matplotlib qt4
from __future__ import division
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

sns.set_style("ticks", {"legend.frameon": True})
mpl.rcParams['axes.color_cycle'] = ['#02A5F4', 'orange', 'green']

In [10]:
def func_learn(x, r=4, s=0):
    return 1 - (1 - s)*np.exp(-r*x)
def func_forget(x, r=4, s=1):
    return s * np.exp(-r * x)

### Styles

In [3]:
line_dashed = {'color': '#000000', 'linewidth': '0.8', 'linestyle': 'dashed'}
text_label = {'color': '#000000', 'fontsize': 11, 'fontstyle': 'italic'}

### Learning and Forgetting Curve

In [21]:
points = np.arange(0, 1, 0.001)
frame, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(7, 3), dpi=160)

ax1.plot(points, func_learn(points))
ax2.plot(points, func_forget(points))

ax1.set_xlabel('Time')
ax1.set_ylabel('Memory strength')
ax2.set_xlabel('Time')

ax1.plot([0, 1], [1, 1], **line_dashed)
ax2.plot([0, 1], [1, 1], **line_dashed)

ax1.annotate('$K_{max}$', xy=(1, 0.5), xytext=(0.8, 1.07), **text_label)
ax2.annotate('$K_{max}$', xy=(1, 0.5), xytext=(0.8, 1.07), **text_label)

ax1.get_xaxis().set_ticks([])
ax2.get_xaxis().set_ticks([])

plt.ylim([0, 1.15])

sns.despine()
plt.tight_layout()

### Knowledge Levels

In [22]:
points = np.arange(0, 1.33, 0.001)

def func(x, r=5):
    if x > 0.5:
        return func_learn(0.5, r=r) * np.exp(-(r-1) * (x - 0.5))
    return func_learn(x, r=r)

plt.figure(num=None, figsize=(6, 3), dpi=160)
frame, = plt.plot(points, [func(x) for x in points])

plt.plot([0, 1.33], [0.8, 0.8], **line_dashed)
plt.plot([0, 1.33], [0.6, 0.6], **line_dashed)
plt.plot([0, 1.33], [0.4, 0.4], **line_dashed)
plt.plot([0, 1.33], [0.2, 0.2], **line_dashed)

plt.annotate('Atomaticity', xy=(1, 0.8), xytext=(1, 0.85), **text_label)
plt.annotate('Recall', xy=(1, 0.6), xytext=(1, 0.65), **text_label)
plt.annotate('Recognition', xy=(1, 0.4), xytext=(1, 0.45), **text_label)
plt.annotate('Femiliarity', xy=(1, 0.2), xytext=(1, 0.25), **text_label)

plt.xlabel('Time')
plt.ylabel('Memory strength')
frame.axes.get_xaxis().set_ticks([])
sns.despine()
plt.tight_layout()